<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping: Solutions

_Author: Joseph Nelson (DC)_

---

## Before Class

#### Install Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`


#### Install GeckoDriver

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

#### Install Firefox

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

## Learning Objectives
- Revisit how to locate elements on a webpage
- Aquire unstructure data from the internet using Beautiful soup.
- Discuss limitations associated with simple requests and urllib libraries
- Introduce Selenium as a solution, and implement a scraper using selenium

## Lesson Guide

- [Introduction](#intro)
- [Building a web scraper](#building-scraper)
- [Retrieving data from the HTML page](#retrieving-data)
    - [Retrieving the restaurant names](#retrieving-names)
    - [Challenge: Retrieving the restaurant locations](#retrieving-locations)
    - [Retrieving the restaurant prices](#retrieving-prices)
    - [Retrieving the restaurant number of bookings](#retrieving-bookings)


- [Introducting Selenium](#selenium)
    - [Running JavaScript before scraping](#selenium-js)
    - [Using regex to only get digits](#selenium-regex)
    - [Challenge: Use Pandas to create a DataFrame of bookings](#challenge-pandas)
    - [Auto-typing using Selenium](#selenium-typing)


- [Summary](#summary)

<a id="intro"></a>
## Introduction

In this codealong lesson, we'll build a web scraper using urllib and BeautifulSoup. We will also explore how to use a headless browser called Selenium.

We'll begin by scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

---

<a id="building-scraper"></a>
## Building a web scraper

Now, let's build a web scraper for OpenTable using urllib and Beautiful Soup:

In [1]:
# import our necessary first packages
from bs4 import BeautifulSoup
import requests

In [54]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = requests.get(url)

At this point, what is in html?

In [55]:
# .text returns the request content in Unicode
html.text[:500]

'           <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title>    <meta  name="robots" content="noindex" > </meta><link  rel="canonical" href="https://www.opentable.com/washington-dc-restaurant-listings" > </link>      <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="/'

We will need to convert this html objct into a soup object so we can parse it using python and BS4

In [4]:
# convert this into a soup object
soup = BeautifulSoup(html.text, 'html.parser')

<a id="retrieving-data"></a>
### Retrieving data from the HTML page

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Hint: We need to know where in the **HTML** the restaurant element is housed.) In order to find the HTML that renders the restaurant location, we can use Google Chrome's Inspect tool:

> http://www.opentable.com/washington-dc-restaurant-listings

> 1. Visit the URL above. 

> 2. Right-click on an element you are interested in, then choose Inspect (in Chrome). 

> 3. This will open the Developer Tools and show the HTML used to render the selected page element. 

> Throughout this lesson, we will use this method to find tags associated with elements of the page we want to scrape.

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [5]:
# print the restaurant names
soup.find_all(name='span', attrs={'class':'rest-row-name-text'})

[<span class="rest-row-name-text">Osinski Shoals</span>,
 <span class="rest-row-name-text">Jamesons</span>,
 <span class="rest-row-name-text">Lind</span>,
 <span class="rest-row-name-text">Ipsum Welch</span>,
 <span class="rest-row-name-text">Animi Shanahan</span>,
 <span class="rest-row-name-text">Non</span>,
 <span class="rest-row-name-text">Altenwerth Plaza</span>,
 <span class="rest-row-name-text">Camdens</span>,
 <span class="rest-row-name-text">Clares</span>,
 <span class="rest-row-name-text">Montes</span>,
 <span class="rest-row-name-text">Dolor Bashirian</span>,
 <span class="rest-row-name-text">Aditya Harris</span>,
 <span class="rest-row-name-text">Daphne Kris</span>,
 <span class="rest-row-name-text">Burg</span>,
 <span class="rest-row-name-text">343 Smitham</span>,
 <span class="rest-row-name-text">Rhiannas</span>,
 <span class="rest-row-name-text">Eileen Light</span>,
 <span class="rest-row-name-text">Jeralds</span>,
 <span class="rest-row-name-text">Larissa Marvin</span>,

It is important to always keep in mind the data types that were returned. Note this is a `list`, and we know that immediately by observing the outer square brackets and commas separating each tag.

Next, note the elements of the list are `Tag` objects, not strings. (If they were strings, they would be surrounded by quotes.) The Beautiful Soup authors chose to display a `Tag` object visually as a text representation of the tag and its contents. However, being an object, it has many methods that we can call on it. For example, next we will use the `encode_contents()` method to return the tag's contents encoded as a Python string.

<a id="retrieving-names"></a>
#### Retrieving the restaurant names

Now that we found a list of tags containing the restaurant names, let's think how we can loop through them all one-by-one. In the following cell, we'll print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [6]:
# for each element you find, print out the restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print(entry.text)

Osinski Shoals
Jamesons
Lind
Ipsum Welch
Animi Shanahan
Non
Altenwerth Plaza
Camdens
Clares
Montes
Dolor Bashirian
Aditya Harris
Daphne Kris
Burg
343 Smitham
Rhiannas
Eileen Light
Jeralds
Larissa Marvin
Fuga Road
302 Jenkins
Lock
Nulla Mountain
Enim Fields
Randis
Club
Facere Heights
1194 Nienow
Modi Valley
Abshire Mountain
Lucile Road
Douglas Islands
1387 Kuphal
Emerson Ankunding
Martas
Drive
Jewels
Joseph Fall
Katlyns
Optio Stroman
Agloe Bar & Grill
Voluptatem
Chaims
Williams
Ben Jaskolski
Deleniti
Destany Island
Rapid
Iste Prairie
Kohler
Velit Knoll
Olens
Meadow
Raymond Barrows
Nihil Creek
Dolor
1487 Runte
Vitae Square
1027 Blick
Aglae Crooks
Temporibus
Nicholes
Rosella Pfeffer
Florencio Schroeder
Ab O'Kon
Nellies
272 Morissette
Quo
Kurt Ziemann
Sint Grove
Corrupti
Koch
Doloribus
Erdman Drives
767 Johnson
Haven
Sit Greenfelder
Cobys
Phyllis Parker
Accusamus Tunnel
Drive
Ipsam
In
Hazle Dicki
Doloremque Hermann
Polly Leannon
O'Reilly
343 Considine
Lillie Bartell
Lindsays
Orvilles
Keys


Great!

<a id="retrieving-locations"></a>
#### Challenge: Retrieving the restaurant locations

Can you repeat that process for finding the location? For example, barmini by Jose Andres is in the location listed as "Penn Quarter" in our search results.

In [7]:
# first, see if you can identify the location for all elements -- print it out
soup.find_all('span', {'class':'rest-row-meta--location rest-row-meta-text'})

[<span class="rest-row-meta--location rest-row-meta-text">Robelborough</span>,
 <span class="rest-row-meta--location rest-row-meta-text">West Tillmanfurt</span>,
 <span class="rest-row-meta--location rest-row-meta-text">West Ellenchester</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Marinashire</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Strosinmouth</span>,
 <span class="rest-row-meta--location rest-row-meta-text">North Orland</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Catherineview</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Beverlyberg</span>,
 <span class="rest-row-meta--location rest-row-meta-text">East Mathewport</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Kozeyland</span>,
 <span class="rest-row-meta--location rest-row-meta-text">South Jackelinehaven</span>,
 <span class="rest-row-meta--location rest-row-meta-text">Trentonville</span>,
 <span class="rest-row-meta--locatio

In [8]:
# now print out EACH location for the restaurants
for entry in soup.find_all('span', {'class':'rest-row-meta--location rest-row-meta-text'}):
    print(entry.text)

Robelborough
West Tillmanfurt
West Ellenchester
Marinashire
Strosinmouth
North Orland
Catherineview
Beverlyberg
East Mathewport
Kozeyland
South Jackelinehaven
Trentonville
New Margie
Lake Wymanmouth
East Tarastad
Strackeville
Danielfort
Gaylordstad
Eddfort
New Elton
Elinoreside
South Sedrick
Fritschville
Keventon
Lake Hattie
Deckowburgh
Monaborough
East Vivianne
Port Cicero
Berthachester
McLaughlinshire
Lake Okey
Murrayton
East Jeromehaven
Stoltenbergside
Doyleland
Montyfort
Aufderharhaven
North Beulah
North Elta
Irmafort
Port Mossiemouth
Bobbyborough
Lake Pamela
South Juwantown
New Jarrell
East Fiona
West Carmenmouth
Weberport
North Robyn
South Rosemary
Port Jeromebury
Luisastad
West Novella
Lake Orietown
Lake Duaneshire
West Robert
Wintheiserstad
Wisokyborough
South Marjolaine
East Briceborough
South Beryl
East Nikitaville
Murphyshire
Port Tatyanafort
Sauertown
Hagenesview
Hazelton
Kingberg
East Donny
Langoshland
East Robertochester
North Nyah
East Camdenmouth
Majorberg
Halliefurt
Ne

<a id="retrieving-prices"></a>
#### Retrieving the restaurant prices

Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [9]:
# print out all prices
soup.find_all('div', {'class':'rest-row-pricing'})

[<div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $        </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    $  </i> <span class="pricing--not-the-price"> </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $        </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $ 

In [10]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print(entry.find('i').text)

  $    $      
  $    $      
  $    $    $    
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    
  $    $    $    
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    $  
  $    

That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [11]:
# print the number of dollars signs per restaurant
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    price = entry.find('i').text
    print(price.count('$'))

2
2
3
4
2
3
3
2
4
4
4
2
4
4
4
4
2
2
4
3
4
4
4
2
2
2
2
4
2
3
4
4
3
3
2
4
3
4
2
2
2
2
3
2
3
4
4
2
3
3
3
4
2
3
4
3
4
3
3
3
2
2
3
4
3
3
4
4
2
4
2
4
4
4
3
3
2
4
4
2
4
2
3
2
4
3
4
2
2
4
3
3
4
4
2
3
4
4
2
3


Phew, nice work. 

<a id="retrieving-bookings"></a>
#### Retrieving the restaurant number of bookings

One more, right? We only need to find the number times a restaurant was booked. In the next cell, print out all objects that contain the number of times the restaurant was booked.

In [12]:
# print out all objects that contain the number of times the restaurant was booked
soup.find_all('div', {'class':'booking'})

[]

That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [21]:
# let's first try printing out all 'div' objects
#  NOTE: This is a too many objects to store in this notebook!
#        So, uncomment the code below to run it.

#for entry in soup.find_all('div'):
#    print(entry)

I still don't see it. Let's search our entire soup object:

In [22]:
# print out soup, do command+f for "booked ". 
#   Uncomment the below to run.

# soup

 <!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="IE=9; IE=8; IE=7; IE=EDGE" http-equiv="X-UA-Compatible"/> <title>Restaurant Reservation Availability</title> <meta content="noindex" name="robots"/> <link href="https://www.opentable.com/washington-dc-restaurant-listings" rel="canonical"/> <link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-16.png" rel="icon" sizes="16x16"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-32.png" rel="icon" sizes="32x32"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-48.png" rel="icon" sizes="48x48"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-64.png" rel="icon" sizes="64x64"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-128.png" rel="icon" sizes="

What do you notice? Why is this happening?

<a id="selenium"></a>
## Introducing Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

In [23]:
# import
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

In [35]:
# STOP
# what is going to happen when I run the next cell?

In [24]:
# create a driver called Firefox
driver = webdriver.Firefox()

Pretty crazy, right? Let's close that driver.

In [25]:
# close it
driver.close()

In [26]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Firefox()
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [27]:
# visit our OpenTable page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

In [28]:
driver.title

'Washington, D.C. Area Restaurants List | OpenTable'

In [29]:
driver.close()

<a id="selenium-js"></a>
### Running JavaScript before scraping

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [30]:
# import sleep
from time import sleep

In [31]:
# visit our relevant page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# wait one second
sleep(1)

#grab the page source
html = driver.page_source

**Pop Quiz:** What do we need to do with this HTML?

In [32]:
# BeautifulSoup it!
html = BeautifulSoup(html, "lxml")

Now, let's return to our earlier problem: How do we locate bookings on the page?

In [33]:
# print out the number bookings for all restaurants
html.find_all('div', {'class':'booking'})

[<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 482 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 219 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 200 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 68 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 146 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 45 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 118 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 25 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 36 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 21 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 30 times today</div>,
 <div class="booking"><span class="t

In [34]:
# now print out each booking for the listings using a loop
for entry in html.find_all('div', {'class':'booking'}):
    print(entry)

<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 482 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 219 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 200 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 68 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 146 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 45 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 118 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 25 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 36 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 21 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 30 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 21 

Let's grab just the text of each of these entries.

In [35]:
# do the same as above, but grabbing only the text content
for entry in html.find_all('div', {'class':'booking'}):
    print(entry.text)

Booked 2 times today
Booked 482 times today
Booked 219 times today
Booked 200 times today
Booked 68 times today
Booked 146 times today
Booked 45 times today
Booked 118 times today
Booked 25 times today
Booked 36 times today
Booked 21 times today
Booked 30 times today
Booked 21 times today
Booked 36 times today
Booked 26 times today
Booked 56 times today
Booked 16 times today
Booked 50 times today
Booked 33 times today
Booked 29 times today
Booked 100 times today
Booked 31 times today
Booked 54 times today
Booked 26 times today
Booked 62 times today
Booked 59 times today
Booked 59 times today
Booked 27 times today
Booked 22 times today
Booked 37 times today
Booked 73 times today
Booked 65 times today
Booked 14 times today
Booked 19 times today
Booked 58 times today
Booked 7 times today
Booked 19 times today
Booked 4 times today
Booked 27 times today
Booked 21 times today
Booked 6 times today
Booked 10 times today
Booked 5 times today
Booked 49 times today
Booked 58 times today
Booked 56

In [36]:
driver.close()

We've succeeded!

<a id="selenium-regex"></a>
### Using regex to only get digits

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [37]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [38]:
# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search(r'\d+', booking.text)
    
    if match:
        # print if found
        print(match.group())
    else:
        # otherwise pass
        pass

2
482
219
200
68
146
45
118
25
36
21
30
21
36
26
56
16
50
33
29
100
31
54
26
62
59
59
27
22
37
73
65
14
19
58
7
19
4
27
21
6
10
5
49
58
56
15
49
28
42
21
17
28
3
9
41
8
4
7
7
13
10
24
21
33
6
23
13
14
14
35
14
29
4
13
2
5
32
13
10
6
5
4
3
19
3
18
33
19
9
6
6
47
17
14
33
8
19
6


Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [40]:
# print out all entries
#   NOTE: Has many entries. Uncomment the below code to run it!

# soup.find_all('div', {'class':'result content-section-list-row cf with-times'})

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [41]:
# We want to only retrieve the text of the bookings.
# But what would happen if we just naively print the text of each node?

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print(entry.find('div', {'class':'booking'}))   # try adding .text

<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>
None
<div class="booking"><span class="tadpole"></span>Booked 482 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 219 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 200 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 68 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 146 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 45 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 118 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 25 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 36 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 21 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 30 times today</div>
<div class="booking"><span class="tadpole"></span>Booke

In [42]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
        divs = entry.find('div', {'class':'booking'})
        
        if divs:
            print(divs.text)
        else:
            print('ZERO')

Booked 2 times today
ZERO
Booked 482 times today
Booked 219 times today
Booked 200 times today
Booked 68 times today
Booked 146 times today
Booked 45 times today
Booked 118 times today
Booked 25 times today
Booked 36 times today
Booked 21 times today
Booked 30 times today
Booked 21 times today
Booked 36 times today
Booked 26 times today
Booked 56 times today
Booked 16 times today
Booked 50 times today
Booked 33 times today
Booked 29 times today
Booked 100 times today
Booked 31 times today
Booked 54 times today
Booked 26 times today
Booked 62 times today
Booked 59 times today
Booked 59 times today
Booked 27 times today
Booked 22 times today
Booked 37 times today
Booked 73 times today
Booked 65 times today
Booked 14 times today
Booked 19 times today
Booked 58 times today
Booked 7 times today
Booked 19 times today
Booked 4 times today
Booked 27 times today
Booked 21 times today
Booked 6 times today
Booked 10 times today
Booked 5 times today
Booked 49 times today
Booked 58 times today
Book

What do you notice takes the place when booking is not found?

We could use exception handling (`try`/`except` blocks) to resolve this. However, exceptions should only be used to handle rare or unexpected errors -- never for normal program flow.

In this case, we expect that some entries will be zero. So, we can just use an `if` statement that tests whether there are any `divs` present; if not, display `'ZERO'`. Here's a demo:

In [43]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    
    if booking_tag:
        print(booking_tag.text)
    else:
        print('ZERO')

Booked 2 times today
ZERO
Booked 482 times today
Booked 219 times today
Booked 200 times today
Booked 68 times today
Booked 146 times today
Booked 45 times today
Booked 118 times today
Booked 25 times today
Booked 36 times today
Booked 21 times today
Booked 30 times today
Booked 21 times today
Booked 36 times today
Booked 26 times today
Booked 56 times today
Booked 16 times today
Booked 50 times today
Booked 33 times today
Booked 29 times today
Booked 100 times today
Booked 31 times today
Booked 54 times today
Booked 26 times today
Booked 62 times today
Booked 59 times today
Booked 59 times today
Booked 27 times today
Booked 22 times today
Booked 37 times today
Booked 73 times today
Booked 65 times today
Booked 14 times today
Booked 19 times today
Booked 58 times today
Booked 7 times today
Booked 19 times today
Booked 4 times today
Booked 27 times today
Booked 21 times today
Booked 6 times today
Booked 10 times today
Booked 5 times today
Booked 49 times today
Booked 58 times today
Book

After previously completing this, we observed that all other elements WILL be returned. This means we do not have to always handle these cases.

<a id="challenge-pandas"></a>
### Challenge: Use Pandas to create a DataFrame of bookings

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [44]:
import pandas as pd

In [45]:
# I'm going to create my empty df first
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [46]:
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    # grab the name
    name = entry.find('span', {'class':'rest-row-name-text'}).text
    
    # grab the location 
    location = entry.find('span', {'class':'rest-row-meta--location rest-row-meta-text'}).text
    
    # grab the price
    price = entry.find('div', {'class':'rest-row-pricing'}).find('i').text.count('$')
    
    # try to find the number of bookings
    bookings = 'NA'
    booking_tag = entry.find('div', {'class':'booking'})
    if booking_tag:
        match = re.search('\d+', booking_tag.text)
        
        if match:
            bookings = match.group()
    
    dc_eats.loc[len(dc_eats)]=[name, location, price, bookings]

In [47]:
# check out our work
dc_eats.head()

,name,location,price,bookings
0,Ruffino's - Arlington,Arlington,2,2
1,Joe's Place Pizza and Pasta,Arlington,2,NA
2,Founding Farmers - DC,Foggy Bottom,2,482
3,Farmers Fishers Bakers,Georgetown,2,219
4,Filomena Ristorante,Georgetown,3,200


Awesome! We succeeded.

<a id="selenium-typing"></a>
### Auto-typing using Selenium

Now, let's explore some of the other functionality of a webdriver. We've barely scratched the surface.

In [48]:
# we can send keys as well
# import
from selenium.webdriver.common.keys import Keys

In [49]:
# open Firefox
driver = webdriver.Firefox()
# visit Python
driver.get("http://www.python.org")
# verify we're in the right place
assert "Python" in driver.title

Let's try automatedly typing `pycon` in the search box and hitting the return key:

In [50]:
# find the search position
elem = driver.find_element_by_name("q")

# clear it
elem.clear()

# type in pycon
elem.send_keys("pycon")

# send those keys
elem.send_keys(Keys.RETURN)

In [51]:
# close
driver.close()

In [52]:
# all at once:
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title

elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
driver.close()

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.